In [ ]:
import json


StatementMeta(, , -1, SessionError, , SessionError)

AVAILABLE_WORKSPACE_CAPACITY_EXCEEDED: Livy session has failed. Session state: Error. Error code: AVAILABLE_WORKSPACE_CAPACITY_EXCEEDED. Your job requested 12 vcores. However, the workspace only has 4 vcores available out of quota of 12 vcores for node size family [MemoryOptimized]. Try ending the running job(s), reducing the numbers of vcores requested or increasing your vcore quota. https://learn.microsoft.com/en-us/azure/synapse-analytics/spark/apache-spark-concepts#workspace-level Source: User.

In [ ]:
output_data= json.loads(bronze_output)

start_date=output_data.get("start_date")
silver_adls=output_data.get("silver_adls")
bronze_adls=output_data.get("bronze_adls")

In [19]:
# from datetime import date, timedelta
# start_date=date.today()-timedelta(1)
# bronze_adls="abfss://bronze@synonlypunya.dfs.core.windows.net/"
# silver_adls="abfss://silver@synonlypunya.dfs.core.windows.net/"

StatementMeta(synapseonly, 1, 2, Finished, Available, Finished)

In [23]:
from pyspark.sql.functions import col,isnull,when
from pyspark.sql.types import TimestampType
df=spark.read.option("multiline","true").json(f"{bronze_adls}{start_date}_earthquake_data.json")

StatementMeta(synapseonly, 1, 6, Finished, Available, Finished)

In [24]:
# Reshape earthquake data
df = (
    df
    .select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
    )
)

StatementMeta(synapseonly, 1, 7, Finished, Available, Finished)

In [25]:
# Validate data: Check for missing or null values
df = (
    df
    .withColumn('longitude', when(isnull(col('longitude')), 0).otherwise(col('longitude')))
    .withColumn('latitude', when(isnull(col('latitude')), 0).otherwise(col('latitude')))
    .withColumn('time', when(isnull(col('time')), 0).otherwise(col('time')))
)

StatementMeta(synapseonly, 1, 8, Finished, Available, Finished)

In [28]:

df.show(1,0)

StatementMeta(synapseonly, 1, 11, Finished, Available, Finished)

+----------+------------+----------+---------+--------------------------------------+------------------------------+---+----+-------+-------------+-------------+
|id        |longitude   |latitude  |elevation|title                                 |place_description             |sig|mag |magType|time         |updated      |
+----------+------------+----------+---------+--------------------------------------+------------------------------+---+----+-------+-------------+-------------+
|ci40174562|-116.3943333|33.3758333|9.55     |M 0.7 - 14 km N of Borrego Springs, CA|14 km N of Borrego Springs, CA|7  |0.68|ml     |1738195018480|1738195304790|
+----------+------------+----------+---------+--------------------------------------+------------------------------+---+----+-------+-------------+-------------+



In [29]:
# Convert 'time' and 'updated' to timestamp from Unix time
df = (
    df
    .withColumn('time', (col('time') / 1000).cast(TimestampType()))
    .withColumn('updated', (col('updated') / 1000).cast(TimestampType()))
)

StatementMeta(synapseonly, 1, 12, Finished, Available, Finished)

In [31]:
silver_data=f"{silver_adls}earthquake_events_silver/"

StatementMeta(synapseonly, 1, 14, Finished, Available, Finished)

In [32]:
df.write.mode('append').parquet(silver_data)
     

StatementMeta(synapseonly, 1, 15, Finished, Available, Finished)

In [ ]:
''' Data Factory
dbutils.notebook.exit(silver_output_path)
'''